In [1]:
%pip install ..

ERROR: Directory '..' is not installable. Neither 'setup.py' nor 'pyproject.toml' found.

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

device = torch.device("cuda:0")

In [2]:
from spuco.utils import set_seed

set_seed(0)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [3]:
from spuco.datasets import SpuCoMNIST, SpuriousFeatureDifficulty

classes = [[0, 1], [2, 3], [4, 5], [6, 7], [8, 9]]
difficulty = SpuriousFeatureDifficulty.MAGNITUDE_LARGE

valset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    classes=classes,
    split="val"
)
valset.initialize()


Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:02<00:00, 3760807.10it/s]


Extracting /data/mnist/MNIST\raw\train-images-idx3-ubyte.gz to /data/mnist/MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 440822.04it/s]


Extracting /data/mnist/MNIST\raw\train-labels-idx1-ubyte.gz to /data/mnist/MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 3845212.42it/s]


Extracting /data/mnist/MNIST\raw\t10k-images-idx3-ubyte.gz to /data/mnist/MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 161297.53it/s]


Extracting /data/mnist/MNIST\raw\t10k-labels-idx1-ubyte.gz to /data/mnist/MNIST\raw



100%|██████████| 11996/11996 [00:01<00:00, 8952.08it/s]


In [4]:
from spuco.robust_train import ERM
from spuco.datasets import SpuCoMNIST, SpuriousFeatureDifficulty
import torchvision.transforms as T

classes = [[0, 1], [2, 3], [4, 5], [6, 7], [8, 9]]
difficulty = SpuriousFeatureDifficulty.MAGNITUDE_LARGE

trainset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    spurious_correlation_strength=0.99,
    classes=classes,
    split="train"
)
trainset.initialize()

testset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    classes=classes,
    split="test"
)
testset.initialize()


100%|██████████| 10000/10000 [00:00<00:00, 10363.09it/s]


In [6]:
print(len(valset))
print(len(trainset))
print(len(testset))

11996
48004
10000


In [7]:
T.ToPILImage()(trainset[1000][0]).resize((28,28))

In [8]:
from spuco.models import model_factory 

model = model_factory("resnet18", trainset[0][0].shape, trainset.num_classes).to(device)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\bryan/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:02<00:00, 18.2MB/s]


In [ ]:
### Simple Model

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        # Example architecture
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        return self.fc2(x)

In [9]:
from torch.optim import SGD

erm = ERM(
    model=model,
    num_epochs=1,
    trainset=trainset,
    batch_size=64,
    optimizer=SGD(model.parameters(), lr=1e-2, momentum=0.9, nesterov=True),
    device=device,
    verbose=True
)

In [10]:
erm.train()

Epoch 0:   0%|          | 0/751 [00:00<?, ?batch/s]


AttributeError: Can't pickle local object 'SpuCoMNIST.load_data.<locals>.<lambda>'

In [21]:
from spuco.evaluate import Evaluator

evaluator = Evaluator(
    testset=testset,
    group_partition=testset.group_partition,
    group_weights=trainset.group_weights,
    batch_size=64,
    model=model,
    device=device,
    verbose=True
)
evaluator.evaluate()

Evaluating group-wise accuracy:   4%|▍         | 1/25 [00:00<00:12,  1.97it/s]

Group (0, 0) Accuracy: 100.0


Evaluating group-wise accuracy:   8%|▊         | 2/25 [00:01<00:12,  1.85it/s]

Group (0, 1) Accuracy: 0.0


Evaluating group-wise accuracy:  12%|█▏        | 3/25 [00:01<00:12,  1.82it/s]

...

Evaluating group-wise accuracy:  96%|█████████▌| 24/25 [00:13<00:00,  1.90it/s]

Group (4, 3) Accuracy: 1.5151515151515151


Evaluating group-wise accuracy: 100%|██████████| 25/25 [00:13<00:00,  1.81it/s]

Group (4, 4) Accuracy: 100.0


{(0, 0): 100.0,
 (0, 1): 0.0,
 (0, 2): 0.0,
 (0, 3): 0.0,
 (0, 4): 0.0,
 (1, 0): 0.0,
 (1, 1): 100.0,
 (1, 2): 0.0,
 (1, 3): 0.0,
 (1, 4): 0.0,
 (2, 0): 0.0,
 (2, 1): 0.0,
 (2, 2): 100.0,
 (2, 3): 0.0,
 (2, 4): 0.0,
 (3, 0): 0.0,
 (3, 1): 0.0,
 (3, 2): 0.0,
 (3, 3): 100.0,
 (3, 4): 0.0,
 (4, 0): 0.0,
 (4, 1): 0.0,
 (4, 2): 0.0,
 (4, 3): 1.5151515151515151,
 (4, 4): 100.0}

In [22]:
evaluator.worst_group_accuracy

((0, 1), 0.0)

In [23]:
evaluator.average_accuracy

98.99443354467937

In [24]:
evaluator.evaluate_spurious_attribute_prediction()

99.93